In [62]:
import itertools
import json
import math
import os
import random
import re
import sys
from collections import Counter
from datetime import datetime

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import utils
from tqdm import tqdm

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

8. 试卷信息表-testpaper

---

In [63]:
testpaper = pd.read_csv('./data/testpaper.csv')

In [64]:
testpaper = testpaper[(testpaper['type'] == 'testpaper') & (testpaper['status'] != 'draft') ]

9. 试卷结果表-testpaper_result

---

In [65]:
testpaper_result = pd.read_csv('./data/testpaper_result.csv')

C:\Users\Tanphoon\AppData\Local\Temp\ipykernel_22108\4110415055.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  testpaper_result = pd.read_csv('./data/testpaper_result.csv')


In [66]:
testpaper_result = testpaper_result[testpaper_result['type'] == 'testpaper']
testpaper_result = testpaper_result[['userId', 'paperName', 'courseId', 'score']]

融合表8和表9

In [67]:
# 求交集
cap = set(testpaper_result['courseId'].unique()) & set(testpaper['courseId'].unique())
# 表8数据
total_score0 =  testpaper[testpaper['courseId'].isin(cap)][["courseId", "name", "score"]].set_index(["courseId", "name"]) 
# 表9数据
total_score1 = testpaper_result.groupby(['courseId','paperName' ])['score'].max()
# 合并
for idx, score in total_score0.itertuples():
    if idx in total_score1:
        total_score1[idx] = score

In [68]:
user_data = testpaper_result.groupby(['userId', 'courseId', 'paperName']).max()
user_data

score
userId courseId paperName                    
3      2        试卷2                       2.0
       14       《Hadoop大数据基础与应用》期中考试试卷    0.0
       96       课程题目模板V1.2（上传）            0.0
       263      提交项目成果                    0.0
       338      《Hadoop大数据基础与应用》期中考试试卷    0.0
...                                       ...
174581 11109    人工智能（高级）正式卷               0.0
174582 11109    人工智能（高级）正式卷               0.0
174583 11109    人工智能（高级）正式卷               0.0
174584 11109    人工智能（高级）正式卷               0.0
174585 11109    人工智能（高级）正式卷               0.0

[145789 rows x 1 columns]

In [69]:
def get_max_score(row):
    idx = (row.name[1], row.name[2])
    return total_score1[idx] if total_score1[idx] != 0 else 1
user_data['rating'] = user_data.apply(lambda x:int(x['score'] / get_max_score(x) * 10), axis=1)

In [70]:
user_data['total_score'] =  user_data.apply(lambda x:int(get_max_score(x)), axis=1)

In [71]:
user_data['pass_rate'] = user_data['score'] / user_data['total_score'] * 100

In [72]:
user_data = user_data.reset_index()

In [74]:
user_data.pop('paperName')

0                           试卷2
1        《Hadoop大数据基础与应用》期中考试试卷
2                课程题目模板V1.2（上传）
3                        提交项目成果
4        《Hadoop大数据基础与应用》期中考试试卷
                  ...          
47369               人工智能（高级）正式卷
47370               人工智能（高级）正式卷
47371               人工智能（高级）正式卷
47372               人工智能（高级）正式卷
47373               人工智能（高级）正式卷
Name: paperName, Length: 47374, dtype: object

In [75]:
user_data = user_data.groupby(["userId", "courseId"]).max().reset_index()

In [76]:
user_data

,userId,courseId,score,rating,total_score,pass_rate
0,3,2,2.0,10,2,100.0
1,3,14,0.0,0,14,0.0
2,3,96,0.0,0,1,0.0
3,3,263,0.0,0,1,0.0
4,3,338,0.0,0,1,0.0
...,...,...,...,...,...,...
47369,174581,11109,0.0,0,99,0.0
47370,174582,11109,0.0,0,99,0.0
47371,174583,11109,0.0,0,99,0.0
47372,174584,11109,0.0,0,99,0.0


In [77]:
total_score = user_data.groupby('courseId')['total_score'].max()

In [78]:
total_score

courseId
2         2
14       14
96        1
263       1
284      18
         ..
12774    94
12775    90
12776    32
12777     1
12778    95
Name: total_score, Length: 1415, dtype: int64

In [79]:
user_data

,userId,courseId,score,rating,total_score,pass_rate
0,3,2,2.0,10,2,100.0
1,3,14,0.0,0,14,0.0
2,3,96,0.0,0,1,0.0
3,3,263,0.0,0,1,0.0
4,3,338,0.0,0,1,0.0
...,...,...,...,...,...,...
47369,174581,11109,0.0,0,99,0.0
47370,174582,11109,0.0,0,99,0.0
47371,174583,11109,0.0,0,99,0.0
47372,174584,11109,0.0,0,99,0.0


In [80]:
course_score = user_data.groupby('courseId').apply(lambda x: x.set_index('userId')['score'].to_dict()).to_dict()

In [81]:
course_pass_rate =  user_data.groupby('courseId')['score'].mean() / total_score * 100

In [83]:
course_pass_rate.drop(10844).sort_values(ascending=False).to_dict()

{2: 100.0,
 1267: 100.0,
 1933: 100.0,
 1914: 100.0,
 11305: 100.0,
 11334: 100.0,
 8088: 100.0,
 1899: 100.0,
 8087: 100.0,
 1822: 100.0,
 1224: 100.0,
 1819: 100.0,
 8077: 100.0,
 3218: 100.0,
 1750: 100.0,
 2853: 100.0,
 11525: 100.0,
 1725: 100.0,
 2887: 100.0,
 1696: 100.0,
 2073: 100.0,
 1625: 100.0,
 1596: 100.0,
 1553: 100.0,
 2524: 100.0,
 8876: 100.0,
 2209: 100.0,
 11137: 100.0,
 2090: 100.0,
 1084: 100.0,
 1081: 100.0,
 2094: 100.0,
 2095: 100.0,
 2100: 100.0,
 2101: 100.0,
 2108: 100.0,
 2110: 100.0,
 2111: 100.0,
 2112: 100.0,
 2114: 100.0,
 1068: 100.0,
 2118: 100.0,
 2133: 100.0,
 1968: 100.0,
 9015: 100.0,
 2173: 100.0,
 2181: 100.0,
 2187: 100.0,
 10971: 100.0,
 3215: 100.0,
 2072: 100.0,
 4866: 100.0,
 1291: 100.0,
 6159: 100.0,
 5718: 100.0,
 4808: 100.0,
 12097: 100.0,
 3724: 100.0,
 4863: 100.0,
 12030: 100.0,
 4851: 100.0,
 3784: 100.0,
 10609: 100.0,
 1297: 100.0,
 1296: 100.0,
 6491: 100.0,
 5724: 100.0,
 5809: 100.0,
 1491: 100.0,
 447: 100.0,
 7117: 100.0,
 1

In [85]:
用户学习时长情况与预测 = json.load(open("./out/用户学习时长情况与预测.json", "r"))

In [87]:
用户学习时长情况与预测.keys()

dict_keys(['userId_list', 'user_learning_type', 'user_learning_record', 'user_learning_predict'])

In [71]:
json_data = {}

In [72]:
json_data['courseId_list'] = list(course_pass_rate.drop(10844).sort_values(ascending=False).to_dict().keys())

In [ ]:
json_data['userId_list'] = list(user_data['userId'].unique().astype(np.int32))

In [73]:
json_data['course_pass_rate'] = course_pass_rate.drop(10844).sort_values(ascending=False).to_dict()

In [74]:
json.dump(json_data, open("./out/课程通过情况.json", "w"))